# Question: 
- Qual estado possui os clientes com melhores pontuações de crédito?

Nesse notebook iremos Discriminar nossos clientes por estado e gerar insights de acordo com as suas características. Inicialmente, iremos determinar algumas estatisticas descritivas dos grupos para que possamos uma breve análise na pontuação de crédito dos mesmos.

Iremos calcular e entender:
- Média
- Mediana
- Moda
- Desvio padrão

In [3]:
# Import libraries
import pandas as pd
import numpy as np

# Basic Data Exploration 

- Primeiramente vamos explorar o dataset e verificar algumas estatisticas gerais do mesmo.Vamos também verificar porcentage de missing values das variáveis.

In [4]:
# Import dataframe

df = pd.read_csv('desafio1.csv')
display(df.head())
print()
display(df.tail())

,RowNumber,id,sobrenome,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
0,1,e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,6d6e0aa1b9b413e442e2fb68df14b4fc3f91de50,619,SC,F,42,2,0.00,1,1,1
1,2,28dcb083ad90512da16b9430085c2cddb8ca5e12,48e1ad846796fa314f1b4a6702b83343eb5482c5,608,RS,F,41,1,83807.86,1,0,1
2,3,774bc378f787438c9c7594e536787d07a097a54b,f2b29d5d934de615812b697132e767dea0f1e9e2,502,SC,F,42,8,159660.80,3,1,0
3,4,043a71326f7096de155e7f0c559dc62b5e4b7239,85d5c9da7cddd8109ad32a6c348fe2bb10bf99f2,699,SC,F,39,1,0.00,2,0,0
4,5,37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,ddc89c837a6933639de75f28171057060bd322df,850,RS,F,43,2,125510.82,1,1,1


,RowNumber,id,sobrenome,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
6995,6996,3e607c788c09d6e1f99cc50575e471fee41a322a,ef547badb8b0801d06a93155cc052341c749d1c0,633,SC,F,32,1,104001.38,1,0,1
6996,6997,2afdb40334e1bef3451f3db02b12c255c7a9ff57,e3eb2eea41a60b08392da20a49543d05ac484255,623,RS,F,50,2,87116.71,1,1,1
6997,6998,6a0589e36c12f95a8d3678b460519ed654fe0a3e,e65e12a7eb4e0b15e6d3afb127ce38c0a5303188,611,SC,M,68,5,82547.11,2,1,1
6998,6999,c042de909a1fb157765c84c3278d8e6500807367,a52bbf77611eab47a6eff8641a1a9bd01006acbe,666,RS,F,47,5,0.00,1,0,0
6999,7000,cdce83890ea1d7b64b2caaaea76f499cb5dfe684,2265b3657cc5e97382dc46819e3937c363fe165e,521,SC,M,27,4,121325.84,1,1,1


- Vamos estabelecer o id do cliente como index e deletar colunas que não agregam muita informação para a nossa análise. 

In [5]:
# Set id as index
df.set_index('id', inplace = True)
# drop columns
drop_cols = ['sobrenome', 'RowNumber']
df.drop(drop_cols , axis = 1, inplace = True)

df.head(3)

,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
id,,,,,,,,,
e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,619,SC,F,42,2,0.00,1,1,1
28dcb083ad90512da16b9430085c2cddb8ca5e12,608,RS,F,41,1,83807.86,1,0,1
774bc378f787438c9c7594e536787d07a097a54b,502,SC,F,42,8,159660.80,3,1,0


In [6]:
total_rows = df.shape[0]
aux_dic = {'Type':df.dtypes,
      'Missing_values_percent': df.isna().sum()/total_rows}

aux_df = pd.DataFrame(aux_dic)
aux_df

,Type,Missing_values_percent
pontuacao_credito,int64,0.0
estado_residencia,object,0.0
genero,object,0.0
idade,int64,0.0
nivel_estabilidade,int64,0.0
saldo_conta,float64,0.0
numero_produtos,int64,0.0
possui_cartao_de_credito,int64,0.0
membro_ativo,int64,0.0


In [7]:
df.describe()

,pontuacao_credito,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
count,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,649.787000,38.892714,5.010286,76471.883751,1.526000,0.703286,0.512143
std,97.046279,10.465562,2.887500,62403.383911,0.583047,0.456842,0.499888
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,582.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000
50%,651.000000,37.000000,5.000000,97277.130000,1.000000,1.000000,1.000000
75%,717.000000,44.000000,7.250000,127537.155000,2.000000,1.000000,1.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000


In [8]:
df.head(2)

,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
id,,,,,,,,,
e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,619,SC,F,42,2,0.00,1,1,1
28dcb083ad90512da16b9430085c2cddb8ca5e12,608,RS,F,41,1,83807.86,1,0,1


### Pontução de crédito por Genero 

- Homens possuem mais pontuações porém a média mostra que uma aproximidade nos valores. Ou seja, analisando somente as médias pontução não aparenta ter uma certa proximidade. Além disso, também podemos verificar uma pequena vantagem dos homens em relação ao saldo em conta (tanto em quantidades quanto em média) 

In [9]:
df.groupby('genero').agg({'pontuacao_credito': ['count', 'mean'], 'saldo_conta': ['count', 'mean']}).round(0)

pontuacao_credito        saldo_conta         
                   count   mean       count     mean
genero                                              
F                   3209  649.0        3209  75286.0
M                   3791  651.0        3791  77475.0

### Pontuacão por gênero e idade

Vamos separar as idades em grupo de idades para ficar mais claro o publico que estamos lidando (jovens, adultos e idosos). Lembrando que essas determinações dos limites que determinal os grupos é subjetiva e não estão divididas de forma equalitária.
- 18 - 35 - Jovens
- 41-60 - Adultos
- 60+ - Idoso

- Levanto esse ponto em consideração, podemos ver que mulheres adultas possuem uma pequena vantagem com relação aos outros dois grupos do mesmo gênero. Além disso, também obsevamos que as médias de pontuação de idosos e jovens são muito próximas.

- Outro fato relevante é que as médias de creditos de homens adultos e jovens também sao muito próximas uma da outra, porém os idosos levam uma pequena vantagem quando comparado com essas duas. 

In [10]:
df['age_group'] = ['jovem' if age <= 35 else ('adult' if age < 60 else 'idoso') for age in df['idade']]
df.head()

,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo,age_group
id,,,,,,,,,,
e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,619,SC,F,42,2,0.00,1,1,1,adult
28dcb083ad90512da16b9430085c2cddb8ca5e12,608,RS,F,41,1,83807.86,1,0,1,adult
774bc378f787438c9c7594e536787d07a097a54b,502,SC,F,42,8,159660.80,3,1,0,adult
043a71326f7096de155e7f0c559dc62b5e4b7239,699,SC,F,39,1,0.00,2,0,0,adult
37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,850,RS,F,43,2,125510.82,1,1,1,adult


In [11]:
# Grupo de idades
np.sort(df['idade'].unique())

array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 88,
       92])

In [12]:
df.groupby(['genero','age_group']).agg({'pontuacao_credito': ['count','mean'], 'saldo_conta': ['count', 'mean']})

pontuacao_credito             saldo_conta              
                             count        mean       count          mean
genero age_group                                                        
F      adult                  1743  649.836489        1743  76954.572352
       idoso                   173  647.497110         173  74427.373873
       jovem                  1293  647.099768        1293  73152.691678
M      adult                  1986  650.222558        1986  79340.761430
       idoso                   187  656.989305         187  74833.288396
       jovem                  1618  650.758962        1618  75490.991038

### Pontuação por estado

-  Aparentemente o estado PR apresenta uma média maior que os outros estados, que por sua vez, apresentam médias próximas
- Podemos verificar uma proximidade de saldo em conta para homens e mulheres no estado PR porém o mesmo não reflete a realidade dos estados RS e SC, onde homens possuem mais créditos. Com relação aos créditos, os homens possuem maiores médias em todos os estados.
- Caso tenhamos interesse em entender melhor os grupos de idade dos clientes por genero e estado, poremos recorrer ao terceiro dataframe abaixo.Além disso, o nivel de estabilidade também poderia nos gerar insights interessantes sobre cada grupo de cliente, porém não incluirei todas as variáveis na nossa análise.

In [13]:
# estate
df.groupby('estado_residencia').agg({'pontuacao_credito': ['count','mean'], 'saldo_conta': ['count', 'mean']})

pontuacao_credito             saldo_conta               
                              count        mean       count           mean
estado_residencia                                                         
PR                             1731  648.961294        1731  119458.904945
RS                             1750  651.105143        1750   62169.610400
SC                             3519  649.537653        3519   62439.046206

In [14]:
# State and gender
df.groupby(['estado_residencia','genero']).agg({'pontuacao_credito': ['count','mean'], 'saldo_conta': ['count', 'mean']})

pontuacao_credito             saldo_conta  \
                                     count        mean       count   
estado_residencia genero                                             
PR                F                    819  648.570208         819   
                  M                    912  649.312500         912   
RS                F                    791  649.738306         791   
                  M                    959  652.232534         959   
SC                F                   1599  648.067542        1599   
                  M                   1920  650.761979        1920   

                                         
                                   mean  
estado_residencia genero                 
PR                F       119099.619292  
                  M       119781.552917  
RS                F        60275.715917  
                  M        63731.727748  
SC                F        60271.110782  
                  M        64244.529927

In [15]:
# State and gender and age group
df.groupby(['estado_residencia','genero','age_group']).agg({'pontuacao_credito': ['count','mean'], 'saldo_conta': ['count', 'mean']})

pontuacao_credito             saldo_conta  \
                                               count        mean       count   
estado_residencia genero age_group                                             
PR                F      adult                   478  648.161088         478   
                         idoso                    39  648.307692          39   
                         jovem                   302  649.251656         302   
                  M      adult                   510  649.954902         510   
                         idoso                    38  641.815789          38   
                         jovem                   364  649.195055         364   
RS                F      adult                   445  651.496629         445   
                         idoso                    39  657.487179          39   
                         jovem                   307  646.205212         307   
                  M      adult                   499  654.026052         499   
                         idoso                    58  647.465517          58   
                         jovem                   402  650.694030         402   
SC                F      adult                   820  649.912195         820   
                         idoso                    95  643.063158          95   
                         jovem                   684  646.551170         684   
                  M      adult                   977  648.419652         977   
                         idoso                    91  669.395604          91   
                         jovem                   852  651.457746         852   

                                                   
                                             mean  
estado_residencia genero age_group                 
PR                F      adult      119291.580628  
                         idoso      119324.926923  
                         jovem      118766.690430  
                  M      adult      118945.561137  
                         idoso      121552.491316  
                         jovem      120767.981896  
RS                F      adult       64150.143034  
                         idoso       60640.548974  
                         jovem       54613.342769  
                  M      adult       65450.443868  
                         idoso       67742.942586  
                         jovem       61019.564055  
SC                F      adult       59223.939537  
                         idoso       61655.601263  
                         jovem       61334.201170  
                  M      adult       65761.253357  
                         idoso       59843.292198  
                         jovem       62975.367770

# Desafio

Previamente respondemos algumas questões que eu estava em analisar. Agora iremos para as respostas do desafio-1


### - Qual estado possui os clientes com melhores pontuações de crédito?

- Média
- Mediana
- Moda
- Desvio padrão

Nessa caso, não podemos aplicar o groupby para conseguir todas as estatisticas que procuramos de uma vez  devido ao fato de que não é possível associar a moda ao agg. Isso acontece basicamente porque o é metodo mode() retorna uma série e para termos o que queremos devemos além de aplicar o mode a nossa série, também informar o indíce como podemos verificar logo abaixo.

In [16]:
df['pontuacao_credito'].mode()

0    850
dtype: int64

In [17]:
df['pontuacao_credito'].mode()[0]

850

In [18]:
# estate
df.groupby('estado_residencia').agg({'pontuacao_credito': ['mean','median','std']})

pontuacao_credito                  
                               mean median        std
estado_residencia                                    
PR                       648.961294    650  98.607186
RS                       651.105143    650  95.136598
SC                       649.537653    653  97.233493

Para solucionar esse problema, a melhor alternativa é criar uma função que calcula as estatiticas que desejamos e aplica-la num dataframe. 

In [19]:
def calc_stats(df):
    
    '''Calculates discritive statistics of a dataframe
    ARG:
    df(dataframe): The dataframe which the statistic methods will be applied
    RETURNS:
    - a dictionary containing statistics such as mode, median, mean and standard deviation'''
    
    return {'moda':df.mode()[0], 'mediana': df.median(), 'media': df.mean(), 'desvio_padrao':df.std()}

In [20]:
stats_df = df.groupby('estado_residencia')['pontuacao_credito'].apply(calc_stats).unstack()
stats_df

,moda,mediana,media,desvio_padrao
estado_residencia,,,,
PR,850.0,650.0,648.961294,98.607186
RS,850.0,650.0,651.105143,95.136598
SC,850.0,653.0,649.537653,97.233493


In [22]:
# challenger answer

stats_df.to_json('submission.json',orient = 'index')